In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import os

In [11]:
import json
f = open(os.getcwd()+os.sep+'token.json','r')
access = json.load(f)
f.close()

In [46]:
class idealista_search_api():
  
  def __init__(self,access:dict):
    self.token = access['access_token']
    self.token_type = access['token_type']
    self.api_key = 'p4o5n83p24mfdxhpnzkzzo5rpr2xng1s'
    self.country = 'es'
    self.endpoint = f'https://api.idealista.com/3.5/{self.country}/search'
    
  def request(self,payload:dict):
    headers = {
        'Authorization': f'{self.token_type} {self.token}',
        'Content-Type': 'multipart/form-data;'
    }
    payload['apikey'] = self.api_key
    response = requests.post(self.endpoint,params=payload,headers=headers)
    if response.status_code == '200':
      return response
    response.raise_for_status()

In [48]:
idealista = idealista_search_api(access)
payload = {
    'operation':'sale',
    'propertyType':'homes',
    'locale':'es',
    #'locationId':'0-EU-ES-28-07-001-079'
    'locationId':'madrid'
    }
response = idealista.request(payload)

HTTPError: ignored

In [ ]:
def get_property_data(driver:webdriver, url:str) -> pd.DataFrame:
  driver.get(url)
  utag_data = driver.execute_script('return window.utag_data')  # price, size, n_bedrooms, type(indirectly), floor, elevator, garden, terrace, energetic_class, garage, status

  data = {
      'price':int(utag_data.ad_price),
      'size':int(utag_data.ad_characteristics_constructedArea),
      'n_bedrooms':int(utag_data.ad_characteristics_bathNumber),
      'n_bathrooms':int(utag_data.ad_characteristics_roomNumber),
      'has_garage' : int(utag_data.ad_characteristics_hasParking),
      'good_condition': int(utag_data.ad_condition_isGoodCondition),
      'needs_renovating': int(utag_data.ad_condition_isNeedsRenovating),
      'new_development': int(utag_data.ad_condition_isNewDevelopment),
      'energy_cert':utag_data.get('ad_energyCertification_type','not_available'),
      'has_lift': int(utag_data.get('ad_characteristics_hasLift',0)),
      'has_garden': int(utag_data.get('ad_characteristics_hasGarden',0)),
      'has_terrace':int(utag_data.get('ad_characteristics_hasTerrace',0)),
      'has_swimming_pool': int(utag_data.get('ad_characteristics_hasSwimmingPool',0))  
  }

  if 'ad_characteristics_hasLift' in utag_data.leys():
    data['type'] = 'apartment'
  else:
    data['type'] = 'house'

  try:
    tags = driver.find_element(By.CSS_SELECTOR,'div.info-features-tags')
    data['luxury'] = ('lujo' in tags)
  except:
    data['luxury'] = False

    return pd.DataFrame(data).to_csv('idealista_real_estate_data.csv',mode='a')

In [ ]:
url_list = get_properties_url_list(driver,url_target)

for url in url_list:
  get_property_data(driver,url)

NameError: ignored